# imports

In [1]:
import tensorflow as tf

2022-07-24 22:38:00.453141: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-24 22:38:00.453165: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import spektral as spk

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
import glob

# Data Files

In [6]:
abidata = '/root/digi_twin_summer_wireless_dataset/data_split_clean'
abidata2 = '/root/digi_twin_summer_wireless_dataset/dataset-2'
kpi_ = '/*kpis.txt'
tfc_ = '/*traffic.txt'
traincon = '/train'
valcon = '/validate'
testcon = '/test'


In [7]:
trainkpis = glob.glob(abidata+traincon+kpi_)
trainkpis.sort()
traintraffics = glob.glob(abidata+traincon+tfc_)
traintraffics.sort()

valkpis = glob.glob(abidata+valcon+kpi_)
valkpis.sort()
valtraffics = glob.glob(abidata+valcon+tfc_)
valtraffics.sort()

testkpis = glob.glob(abidata+testcon+kpi_)
testkpis.sort()
testtraffics = glob.glob(abidata+testcon+tfc_)
testtraffics.sort()

test2kpis = glob.glob(abidata2+testcon+kpi_)
test2kpis.sort()
test2traffics = glob.glob(abidata2+testcon+tfc_)
test2traffics.sort()

# DataGen

In [8]:
adj_mat = [[0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]]
lap = spk.utils.convolution.gcn_filter(np.array(adj_mat))

In [9]:
class UnoDataGen(tf.keras.utils.Sequence):
    def __init__(self, filenames, hparams):
        self.hparams = hparams
        
        kpis, tris = filenames
        
        
        kpiframe = pd.read_csv(kpis[0], header=None)
        # for k in kpis[1:]:
        #     kpiframe = pd.concat([kpiframe, pd.read_csv(k, header=None)])
        self.kpiframe = kpiframe.reset_index(drop=True)

        self.n = kpiframe.shape[0]
        triframe = pd.read_csv(tris[0], header=None)
        # for k in tris[1:]:
        #     triframe = pd.concat([triframe, pd.read_csv(k, header=None)])
        self.triframe = triframe.reset_index(drop=True)

    def on_epoch_end(self):
        pass
    
    def __getitem__(self, index=None):
        
        a = self.triframe.loc[index]
        b = self.kpiframe.loc[index]
        #links and paths
        f_n_links = 42
        f_n_nodes = 14
        f_n_paths = 10

        f_degrees = [1.0]*14
        
        f_capacities = [1.0]*42

        traffic_in = [float(a[i]) for i in range(0, len(a), 2)]
        traffic_ot = [float(a[i]) for i in range(1, len(a), 2)]
        f_traffic = [traffic_in, traffic_ot]

        f_links_to_paths = [0, 5, 5, 14, 34, 11, 18, 6, 25, 20, 38, 33, 36, 40, 28, 30, 16, 19]

        f_link_state_dim = 32

        f_nodes_to_paths = [0, 1, 1, 7, 10, 2, 5, 3, 6, 4, 9, 10, 9, 11, 12, 13, 5, 4]

        f_node_state_dim = 32

        f_links_to_nodes = [0, 0, 0, 1, 1, 1, 3, 3, 3, 2, 2, 2, 7, 7, 7, 5, 5, 5, 5, 4, 4, 4, 8, 8, 8, 6, 6, 12, 12, 12, 13, 13, 10, 10, 10, 10, 9, 9, 9, 11, 11, 11]

        f_nodes_to_links = [0, 1, 2, 3, 4, 5, 9, 10, 11, 6, 7, 8, 19, 20, 21, 15, 16, 17, 18, 25, 26, 12, 13, 14, 22, 23, 24, 36, 37, 38, 32, 33, 34, 35, 39, 40, 41, 27, 28, 29, 30, 31]

        f_paths_to_x = [0, 0, 1, 1, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9, 9]
        f_sequences_paths_x = [0, 1, 0, 1, 2, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2]

        l_delay = [float(b[i]) for i in range(2, len(b), 3)]

        # features = {
        #     'n_nodes': tf.Variable(tf.constant(f_n_nodes)), #0
        #     'n_links': tf.Variable(tf.constant(f_n_links)), #1
        #     'n_paths': tf.Variable(tf.constant(f_n_paths)), #2
        #     'node_init': tf.Variable(tf.constant(f_degrees)), #3
        #     'link_init': tf.Variable(tf.constant(f_capacities)), #4
        #     'path_init': tf.Variable(tf.constant(f_traffic)), #5
        #     'nodes_to_paths': tf.Variable(tf.constant(f_nodes_to_paths)), #6
        #     'links_to_paths': tf.Variable(tf.constant(f_links_to_paths)), #7
        #     'links_to_nodes': tf.Variable(tf.constant(f_links_to_nodes)), #8
        #     'paths_to_x': tf.Variable(tf.constant(f_paths_to_x)), #9
        #     'sequences_paths_x': tf.Variable(tf.constant(f_sequences_paths_x)), #10
        #     'node_lapacian': tf.Variable(tf.constant(lap, dtype=np.float32)) #11
        # }
        features = [
            tf.Variable(tf.constant(f_degrees)), #0
            tf.Variable(tf.constant(f_capacities)), #1
            tf.Variable(tf.constant(f_traffic)), #2
            tf.Variable(tf.constant(f_n_nodes)), #3
            tf.Variable(tf.constant(f_n_links)), #4
            tf.Variable(tf.constant(f_n_paths)), #5
            tf.Variable(tf.constant(f_nodes_to_paths)), #6
            tf.Variable(tf.constant(f_links_to_paths)), #7
            tf.Variable(tf.constant(f_links_to_nodes)), #8
            tf.Variable(tf.constant(f_paths_to_x)), #9
            tf.Variable(tf.constant(f_sequences_paths_x)), #10
            tf.Variable(tf.constant(adj_mat, dtype=np.float32)), #11
            tf.Variable(tf.constant(f_nodes_to_links)), #12

        ]

        labels = [
            tf.Variable(tf.constant(l_delay)), #0
        ]

        #return sample
        return [features, labels]
    
    def __len__(self):
        return self.n #// self.hparams['batch_size']

In [10]:
hparams = {
    'node_count':14,
    'link_state_dim':32, 
    #[4, 8, 16, 32, 64]
    'path_state_dim':32,
    #[2, 4, 8, 16, 32, 64]
    'node_state_dim':32,
    #[2, 4, 8, 16, 32, 64]
    'T':8,
    'readout_units':8,
    'learning_rate':0.001,
    #[.001, .01, .05]
    'batch_size':10,
    #[8, 16, 32, 64]
    'dropout_rate':0.5,
    #[.5] leave
    'l2':0.1,
    #regulirization constants
    #[.05, .1, .2]
    'l2_2':0.01,
    #[.005, .01, .02]
    'learn_embedding':True, # If false, only the readout is trained
    'readout_layers':2, # number of hidden layers in readout model
    #[2, 3, 4]
}

In [11]:
traingen = UnoDataGen((trainkpis, traintraffics), hparams=hparams)

valgen = UnoDataGen((valkpis, valtraffics), hparams=hparams)

testgen = UnoDataGen((testkpis, testtraffics), hparams=hparams)

test2gen = UnoDataGen((test2kpis, test2traffics), hparams=hparams)

# Models

In [90]:
class Algo1(tf.keras.Model):
    def __init__(self,hparams, output_units=1, final_activation=None):
        super(Algo1, self).__init__()

        self.hparams = hparams
        self.output_units = output_units
        self.final_activation = final_activation
        

        
    def build(self, input_shape=None):
        del input_shape

        #state updaters
        # self.edge_update = tf.keras.layers.GRUCell(hparams['link_state_dim'], name="edge_update")
        self.path_update = tf.keras.layers.GRUCell(hparams['path_state_dim'], name="path_update")
        self.node_update = spk.layers.ECCConv(hparams['node_state_dim'])

        self.edge_update = tf.keras.models.Sequential(name="edge_update")
        for i in range(5):
            self.edge_update.add(tf.keras.layers.Dense(hparams['link_state_dim']))

        #readout-final
        
        self.readout = tf.keras.models.Sequential(name='readout')

        for i in range(hparams['readout_layers']):
            self.readout.add(tf.keras.layers.Dense(hparams['readout_units'], 
                    activation=tf.nn.selu,
                    kernel_regularizer=tf.keras.regularizers.L2(hparams['l2'])))

            self.readout.add(tf.keras.layers.Dropout(rate=hparams['dropout_rate']))

        self.final = tf.keras.layers.Dense(self.output_units, 
                kernel_regularizer=tf.keras.regularizers.L2(hparams['l2_2']),
                activation = self.final_activation )
        
        # self.edge_update.build(tf.TensorShape([None,hparams['path_state_dim']]))
        self.path_update.build(tf.TensorShape([None,hparams['link_state_dim']+hparams['node_state_dim']]))

        self.edge_update.build(tf.TensorShape([None,hparams['link_state_dim']+hparams['node_state_dim']+hparams['path_state_dim']]))

        self.readout.build(input_shape = [None,hparams['path_state_dim']])
        self.final.build(input_shape = [None,hparams['path_state_dim'] + hparams['readout_units'] ])
        self.built = True

    def call(self, inputs, training=False):
        #call == v ==
        f_ = inputs

        #state init
        # shape = tf.stack([f_['n_links'],hparams['link_state_dim']-1], axis=0)
        shape = tf.stack([f_[4],hparams['link_state_dim']-1], axis=0)
        link_state = tf.concat([
            # tf.expand_dims(f_['link_init'],axis=1),
            tf.expand_dims(f_[1],axis=1),
            tf.zeros(shape)
        ], axis=1)

        # shape = tf.stack([f_['n_nodes'],hparams['node_state_dim']-1], axis=0)
        shape = tf.stack([f_[3],hparams['node_state_dim']-1], axis=0)
        node_state = tf.concat([
            # tf.expand_dims(f_['node_init'],axis=1),
            tf.expand_dims(f_[0],axis=1),
            tf.zeros(shape)
        ], axis=1)

        # shape = tf.stack([f_['n_paths'],hparams['path_state_dim']-1], axis=0)
        shape = tf.stack([f_[5],hparams['path_state_dim']-2], axis=0)
        path_state = tf.concat([
            tf.expand_dims(f_[2][0],axis=1),
            tf.expand_dims(f_[2][1],axis=1),
            # tf.expand_dims(f_['path_init'],axis=1),
            # tf.expand_dims(f_[5],axis=1),
            tf.zeros(shape)
        ], axis=1)

        #pull for both
        # paths = f_['paths_to_x']
        paths = f_[9]
        # seqs=  f_['sequences_paths_x']
        seqs=  f_[10]
        # n_paths = f_['n_paths']
        n_paths = f_[5]
        
        for _ in range(hparams['T']):
        #stuff for both
            ids=tf.stack([paths, seqs], axis=1)
            max_len = tf.reduce_max(seqs)+1
            lens = tf.math.segment_sum(data=tf.ones_like(paths),
                                    segment_ids=paths)

            #link stuff
            # h_ = tf.gather(link_state,f_['links_to_paths'])
            h_ = tf.gather(link_state,f_[7])

            shape = tf.stack([n_paths, max_len, hparams['link_state_dim']])
            link_inputs = tf.scatter_nd(ids, h_, shape)
            
            #node stuff
            # h_ = tf.gather(link_state,f_['nodes_to_paths'])
            h_ = tf.gather(link_state,f_[6])

            shape = tf.stack([n_paths, max_len, hparams['node_state_dim']])
            node_inputs = tf.scatter_nd(ids, h_, shape)
            
            x_inputs = tf.concat([link_inputs, node_inputs], axis=2)
            
            #updating path_state
            outputs, path_state = tf.compat.v1.nn.dynamic_rnn(cell = self.path_update,
                                                inputs = x_inputs,
                                                sequence_length = lens,
                                                initial_state = path_state,
                                                dtype=tf.float32)
            
            m = tf.gather_nd(outputs,ids)
            # m = tf.math.unsorted_segment_sum(m, f_['links_to_paths'] ,f_['n_links'])
            m = tf.math.unsorted_segment_sum(m, f_[7] ,f_[4])
            #fitting nodes to links
            # h_ = tf.gather(node_state,f_['links_to_nodes'])
            h_ = tf.gather(node_state,f_[8])

            _con = tf.concat([h_, link_state, m], axis=1)
            link_state = self.edge_update(_con)

            # link_state,_ = self.edge_update(m, [red])
            
            #node update

            # print(tf.gather(link_state, f_[12]))
            # node_state = self.node_update((node_state, f_['adj_mat'], link_state))
            node_state = self.node_update((node_state, f_[11], link_state))

        #readout
                
        if hparams['learn_embedding']:
            r = self.readout(path_state,training=training)
            o = self.final(tf.concat([r,path_state], axis=1))
            
        else:
            r = self.readout(tf.stop_gradient(path_state),training=training)
            o = self.final(tf.concat([r, tf.stop_gradient(path_state)], axis=1) )

        return o

    def train_step(self, data):
        features, labels = data
        
        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            print(tf.math.is_nan(predictions))
            loc  = predictions[...,0]
            delay_prediction = loc
            loss = tf.keras.metrics.mean_squared_error(labels[0], loc)
            print(tf.math.is_nan(loss))

            regularization_loss = sum(self.losses)
            total_loss = loss + regularization_loss
            
        gradients = tape.gradient(total_loss, self.trainable_variables)
        
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        ret = {
            'loss':loss,
            'label/mean/delay':tf.math.reduce_mean(labels[0]),
            'prediction/mean/delay': tf.math.reduce_mean(delay_prediction)
            }
        return ret

    def test_step(self, data):
        features, labels = data
        
        with tf.GradientTape() as tape:
            predictions = self(features, training=False)
            loc  = predictions[...,0]
            delay_prediction = loc
            loss = tf.keras.metrics.mean_squared_error(labels[0], loc)

            regularization_loss = sum(self.losses)
            total_loss = loss + regularization_loss
            
        # gradients = tape.gradient(total_loss, self.trainable_variables)
        
        # self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        ret = {
            'loss':loss,
            'label/mean/delay':tf.math.reduce_mean(labels[0]),
            'prediction/mean/delay': tf.math.reduce_mean(delay_prediction)
            }
        return ret

In [91]:
r_ = Algo1(hparams)
r_.build()
r_.compile(optimizer="adam")

In [92]:
sum(np.isnan(r_(traingen.__getitem__(0)[0])))[0]

/usr/local/lib/python3.8/dist-packages/spektral/layers/convolutional/ecc_conv.py:162: UserWarning: Casting dense adjacency matrix to SparseTensor.This can be an expensive operation. 
  warnings.warn(


0

In [93]:
csv0 = tf.keras.callbacks.CSVLogger(
    '../../Logs/UnoNet0722', separator=',', append=False
)

In [94]:
epochs = 20
loops = 1
r_.fit(
    traingen, 
    steps_per_epoch=traingen.__len__()//epochs, 
    epochs=epochs*loops, 
    validation_data=valgen, 
    validation_steps=valgen.__len__()//epochs, 
    # callbacks=[csv0]
    )

Epoch 1/20


/usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Casting dense adjacency matrix to SparseTensor.This can be an expensive operation. 
  return py_builtins.overload_of(f)(*args)


Tensor("IsNan:0", shape=(None, 1), dtype=bool)
Tensor("IsNan_1:0", shape=(), dtype=bool)


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("concat_1:0", shape=(None,), dtype=int32), values=Tensor("concat:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/algo1_6/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/algo1_6/Reshape_5:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/algo1_6/Reshape_4:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/algo1_6/Cast_2:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/i

Tensor("IsNan:0", shape=(None, 1), dtype=bool)
Tensor("IsNan_1:0", shape=(), dtype=bool)
174/174 [==============================] - 10s 20ms/step - loss: 245546.1863 - label/mean/delay: 429.7069 - prediction/mean/delay: 1.7681 - val_loss: 211338.4531 - val_label/mean/delay: 389.4000 - val_prediction/mean/delay: 5.9817
Epoch 2/20
174/174 [==============================] - 2s 13ms/step - loss: 242099.8451 - label/mean/delay: 429.6686 - prediction/mean/delay: 14.0996 - val_loss: 195716.5312 - val_label/mean/delay: 389.4000 - val_prediction/mean/delay: 27.0336
Epoch 3/20
174/174 [==============================] - 2s 13ms/step - loss: nan - label/mean/delay: 429.7909 - prediction/mean/delay: nan - val_loss: nan - val_label/mean/delay: 389.4000 - val_prediction/mean/delay: nan
Epoch 4/20
174/174 [==============================] - 2s 13ms/step - loss: nan - label/mean/delay: 423.7343 - prediction/mean/delay: nan - val_loss: nan - val_label/mean/delay: 389.4000 - val_prediction/mean/delay: nan

KeyboardInterrupt: 

In [69]:
sum(np.isnan(r_(testgen.__getitem__(0)[0])))[0]>0

True

In [65]:
np.nansum(5)

5

In [31]:
#UnoNet Test1
r_.evaluate(testgen)

1000/1000 [==============================] - 4s 4ms/step - loss: nan - label/mean/delay: 428.4890 - prediction/mean/delay: nan


[492.1000061035156, nan, nan]

In [ ]:
#UnoNet Tetst2
r_.evaluate(test2gen)

1000/1000 [==============================] - 4s 4ms/step - loss: 19958.6998 - label/mean/delay: 431.3978 - prediction/mean/delay: 413.7140


[497.79998779296875, 42391.21484375, 424.42315673828125]